In [1]:
from __future__ import print_function
import pandas as pd
from intermine.webservice import Service
service = Service("http://phytozome.jgi.doe.gov/phytomine/service")

In [2]:
file_gff = '/data2/k821209/Reference/Athaliana_167_gene.gff3'

df_gff = pd.read_csv(file_gff,sep='\t',skiprows=1,header=None)
mask = (df_gff[2] == 'mRNA')
primary_transcript_list = [x.split(';')[1].replace('Name=','') for x in df_gff[mask][8] if x.split(';')[3] == 'longest=1']
genename_list = [x.split('.')[0] for x in primary_transcript_list]

dicGN2PT = dict(zip(genename_list,primary_transcript_list))

In [55]:
file_genelist = 'stress.response.genes.txt'
df_genelist = pd.read_csv(file_genelist,sep='\t',header=None)
df_genelist = df_genelist.set_index(0)
df_genelist.loc['AT5G05600'][3]

' response to salt stress, response to karrikin'

In [48]:
genelist = [dicGN2PT[x] for x in df_genelist.index]

In [18]:
def get_cds(name): # name should be transcript name
    template = service.get_template('Transcript_CDS_sequence')
    rows = template.rows(
        A = {"op": "=", "value": name}
        )
    for row in rows:
        return row["CDSs.sequence.residues"]

In [19]:
def get_desc(name): # genename 
    template = service.get_template('Gene_info')
    rows = template.rows(
            A = {"op": "=", "value": name}
            )
    for row in rows:
        return row["briefDescription"]

In [21]:
file_cluster_ath = '/data2/k821209/tandem_duplication/ver2/Ath_167/Athaliana_167.fa.pep.fa.bp.ev1e5.out7.tandemNnontandem.ver2.gl100.out'
file_cluster_aly = '/data2/k821209/tandem_duplication/ver2/Aly_107/Alyrata_107.fa.pep.fa.bp.ev1e5.out7.tandemNnontandem.ver2.gl100.out'
file_cluster_esa = '/data2/k821209/tandem_duplication/ver2/Tha_173/Thalophila_173.fa.pep.fa.bp.ev1e5.out7.tandemNnontandem.ver2.gl100.out'

In [22]:
df_cluster_ath = pd.read_csv(file_cluster_ath,sep='\t',header=None,skiprows=1)
df_cluster_aly = pd.read_csv(file_cluster_aly,sep='\t',header=None,skiprows=1)
df_cluster_esa = pd.read_csv(file_cluster_esa,sep='\t',header=None,skiprows=1)


In [23]:
mask = (df_cluster_ath[1] == 'whole homologs')
df_cluster_ath = df_cluster_ath[mask]
mask = (df_cluster_aly[1] == 'whole homologs')
df_cluster_aly = df_cluster_aly[mask]
mask = (df_cluster_esa[1] == 'whole homologs')
df_cluster_esa = df_cluster_esa[mask]

In [24]:
df_cluster_ath.head()

,0,1,2,3,4,5
0,Homology cluster 0,whole homologs,AT5G36320.1,AT5G36320.1,33,"AT5G36320.1,AT5G36370.1,AT5G36661.1,AT5G36738...."
4,Homology cluster 1,whole homologs,AT4G20540.1,TKL_IRAK_DUF26-lc.12 - DUF26 kinases have homo...,15,"AT4G20540.1,AT4G20630.1,AT4G20530.1,AT4G20650...."
7,Homology cluster 2,whole homologs,AT3G30385.1,PF05617,12,"AT3G30385.1,AT3G30387.1,AT3G30383.1,AT5G34887...."
12,Homology cluster 3,whole homologs,AT1G19890.1,"histone H3, putative, expressed",12,"AT1G19890.1,AT5G10980.1,AT4G40040.1,AT4G40030...."
17,Homology cluster 4,whole homologs,AT2G35635.1,"ubiquitin family protein, putative, expressed",11,"AT2G35635.1,AT1G31340.1,AT5G03240.1,AT5G20620...."


In [26]:

def get_gene2id(df_cluster):
    dicGene2ID = {}
    for i in df_cluster.index:
        homologlist = df_cluster.loc[i][5].split(',')
        for gene in homologlist:
            try:
                if dicGene2ID[gene]:
                    print (1)
            except :
                dicGene2ID[gene] = df_cluster.loc[i][0]
    return dicGene2ID

In [27]:
dicGene2ID_ath = get_gene2id(df_cluster_ath)
dicGene2ID_aly = get_gene2id(df_cluster_aly)
dicGene2ID_esa = get_gene2id(df_cluster_esa)

In [39]:
file_mcscan_ath2aly = './droughtnet/synteny/At2Al.collinearity.kaks' 
file_mcscan_ath2esa = './droughtnet/synteny/At2Es.collinearity.kaks'

In [40]:
def get_ortholog(file_mcscan):
    dic = {}
    for line in open(file_mcscan):
        if line[0] == '#':
            continue
        cell      = line.strip().split('\t')
        syntenyID = cell[0]
        if cell[1].split('|')[0] == 'ATH':
            geneA     = cell[1].split('|')[1]
            geneB     = cell[2].split('|')[1]
        elif cell[2].split('|')[0] == 'ATH':
            geneA     = cell[2].split('|')[1]
            geneB     = cell[1].split('|')[1]
        ka = cell[4]
        ks = cell[5]
        try:
            dic[geneA].append([geneB,ka,ks])
        except KeyError:    
            dic[geneA] = [[geneB,ka,ks]]
    return dic



In [41]:
dicOrth_ath2aly = get_ortholog(file_mcscan_ath2aly)
dicOrth_ath2esa = get_ortholog(file_mcscan_ath2esa)

In [56]:
#geneATH            = 'AT5G10980.1' 
#geneATH            = 'AT2G35635.1'
def get_orthologs_span(geneATH):
    try:
        geneALY            = dicOrth_ath2aly[geneATH]
        geneALY.sort(key=lambda x:float(x[2]))
        #print geneALY
        geneALY            = geneALY[0][0]

        geneESA            = dicOrth_ath2esa[geneATH]
        geneESA.sort(key=lambda x:float(x[2]))
        #print geneESA
        geneESA            = geneESA[0][0]
    except KeyError:
        return (geneATH,'no orthology')
    else:
        try:
            ID_geneATH         = dicGene2ID_ath[geneATH]
            homologsOfgeneATH  = df_cluster_ath[(df_cluster_ath[0] == ID_geneATH)][5].values[0]
            description        = df_cluster_ath[(df_cluster_ath[0] == ID_geneATH)][3].values[0]
        except KeyError:
            homologsOfgeneATH = 'None'
            description       = 'None'
        try:
            ID_geneALY         = dicGene2ID_aly[geneALY]
            homologsOfgeneALY  = df_cluster_aly[(df_cluster_aly[0] == ID_geneALY)][5].values[0]
        except KeyError:
            homologsOfgeneALY = 'None'
        try:
            ID_geneESA         = dicGene2ID_esa[geneESA]
            homologsOfgeneESA  = df_cluster_esa[(df_cluster_esa[0] == ID_geneESA)][5].values[0]
        except KeyError:
            homologsOfgeneESA = 'None'
        #print '-',geneATH, geneALY, geneESA, description
        #print '---ATH_homologs:',homologsOfgeneATH
        #print '---ALY_homologs:',homologsOfgeneALY
        #print '---ESA_homologs:',homologsOfgeneESA
        return (geneATH, geneALY, geneESA, get_desc(geneATH.split('.')[0]),df_genelist.loc[geneATH.split('.')[0]][3],homologsOfgeneATH,homologsOfgeneALY,homologsOfgeneESA)
        
        

In [58]:
Outfile = open('1.GeneToHomologCluster.out.txt','w')
for gene in genelist:
    result = get_orthologs_span(gene)
    if len(result) > 2:
        print ('\t'.join(map(str,result)),file=Outfile)

In [59]:
Outfile.close()